In [ ]:
import os
import shutil
import subprocess
import sys
import pandas as pd
import numpy as np

# Setup paths
EXPERIMENT_DIR = '/home/code/experiments/001_baseline'
DATA_DIR = '/home/data'
SUBMISSION_PATH = '/home/submission/submission.csv'

# Copy sample submission to experiment dir
shutil.copy(os.path.join(DATA_DIR, 'sample_submission.csv'), os.path.join(EXPERIMENT_DIR, 'submission.csv'))

print("Setup complete.")

In [ ]:
# Compile C++ code
cpp_files = ['tree_packer_v21.cpp', 'bp.cpp']
executables = ['tree_packer_v21.exe', 'bp.exe']

compile_cmds = [
    f"g++ -O3 -march=native -std=c++17 -fopenmp -o {os.path.join(EXPERIMENT_DIR, 'tree_packer_v21.exe')} {os.path.join(EXPERIMENT_DIR, 'tree_packer_v21.cpp')}",
    f"g++ -O3 -std=c++17 -o {os.path.join(EXPERIMENT_DIR, 'bp.exe')} {os.path.join(EXPERIMENT_DIR, 'bp.cpp')}"
]

for cmd in compile_cmds:
    print(f"Compiling: {cmd}")
    subprocess.check_call(cmd, shell=True)

print("Compilation complete.")

In [ ]:
class SantaSolver:
    def __init__(self, experiment_dir, input_file='submission.csv'):
        self.experiment_dir = experiment_dir
        self.input_file = os.path.join(experiment_dir, input_file)
        self.output_file = os.path.join(experiment_dir, 'submission_v21.csv')
        self.packer_exe = os.path.join(experiment_dir, 'tree_packer_v21.exe')
        self.bp_exe = os.path.join(experiment_dir, 'bp.exe')
        
    def optimize(self, iters=1000, restarts=4):
        cmd = f"{self.packer_exe} -i {self.input_file} -o {self.output_file} -n {iters} -r {restarts}"
        print(f"Running: {cmd}")
        
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            shell=True
        )
        
        final_score = None
        for line in process.stdout:
            print(line, end='')
            if "Final:" in line:
                try:
                    final_score = float(line.split()[1])
                except:
                    pass
        
        process.wait()
        if process.returncode != 0:
            raise RuntimeError("Optimization failed")
            
        # Update input file for next step
        if os.path.exists(self.output_file):
            shutil.copy(self.output_file, self.input_file)
            
        return final_score

    def back_propagate(self):
        cmd = f"{self.bp_exe} {self.input_file} {self.input_file}"
        print(f"Running: {cmd}")
        
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            shell=True
        )
        
        final_score = None
        for line in process.stdout:
            print(line, end='')
            if "Final score:" in line:
                try:
                    final_score = float(line.split(":")[1])
                except:
                    pass
        
        process.wait()
        if process.returncode != 0:
            raise RuntimeError("Backward propagation failed")
            
        return final_score

print("SantaSolver class defined.")

In [ ]:
# Run optimization
solver = SantaSolver(EXPERIMENT_DIR)

# Run a few iterations
# For baseline, we'll do 1 pass of optimization and 1 pass of BP
print("Starting optimization...")
score1 = solver.optimize(iters=1000, restarts=4)
print(f"Score after optimization: {score1}")

print("Starting backward propagation...")
score2 = solver.back_propagate()
print(f"Score after BP: {score2}")

final_score = score2 if score2 is not None else score1
print(f"Final Score: {final_score}")

In [ ]:
# Generate submission file
# The solver updates 'submission.csv' in the experiment dir
# We need to copy it to the submission path
if os.path.exists(os.path.join(EXPERIMENT_DIR, 'submission.csv')):
    os.makedirs(os.path.dirname(SUBMISSION_PATH), exist_ok=True)
    shutil.copy(os.path.join(EXPERIMENT_DIR, 'submission.csv'), SUBMISSION_PATH)
    print(f"Submission saved to {SUBMISSION_PATH}")
else:
    print("Error: submission.csv not found!")

# Verify submission format
df = pd.read_csv(SUBMISSION_PATH)
print(df.head())
print(df.columns)